In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [ ]:
DECADE = '1840'
N_CLUSTERS = 5

# Chargement des fichiers de la décennie

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5]

In [ ]:
texts = [open(data_path + f).read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

# Vectorisation du texte

In [ ]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [ ]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
tfidf_vectors

In [ ]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

# Comprendre les vecteurs et leurs "distances"

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
tfidf_array[0]

In [ ]:
texts[0][:200]

In [ ]:
tfidf_array[0]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [ ]:
km_model.fit(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))